# Create sampling points for Rhino and Modelica

Stupid random sampling. More systematic ways: explore pyDOE! 
- https://pythonhosted.org/pyDOE/index.html
- https://github.com/clicumu/pyDOE2

How to identify things

- Project (projectid: Kiss1)
- Study: a variant within the parameters of the project (studyid -> vergeben vom Optimierer oder wie hier)
- Geometry Model (parametric) -> github commit id nutzen
- Geometry Instance -> id von rhino vergeben
- Analysis model (parametric) -> github commit id nutzen
- analysis instance -> von analyse vergeben

geometrie ist auch eine analyse!!

Welche Rolle spielt CAD? nur am ende zur Doku...

### Input parameters (for Rhino)

- number of racks
- number of cells per rack
- cooling channel width
- cooling channel height
- wall thickness

### Input parameters for OpenModelica

- cover (wall between cell and pipe) 
  - area_h = 3.44e-4, 
  - c_wall = 888, 
  - k_wall = 220,
  - rho_wall = 2700, 
  - s = 0.002
- "wall" between two cells
  - area_h = 1.72e-4, 
  - c_wall = 888, 
  - k_wall = 220, 
  - rho_wall = 2700, 
  - s = 0.04
- pipe
  - crossArea = 0.32e-4, 
  - length = 0.043, 
  - perimeter = 0.024, 
- mass flow source
  - m_flow = 0.0037,
- Gc_Thermischer_Kontaktwiderstand(k = 3.311) 



In [7]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from uuid import uuid1
import json
import random


In [26]:
# constants
constants = {
    "C_quartercell": 36.3,
    "k_cell": 3.311, 

    "c_cover": 888.,
    "k_cover": 220.,
    "rho_cover": 2700.,

    "c_plate": 888.,
    "k_plate": 220.,
    "rho_plate": 2700.,

    "channel_dist": 0.04,

    "cell_width": 0.043,
    "stoptime": 2000,
}

# Configurations for number of racks and number of cells. 
# total number of cells in a battery pack is 2*nracks*ncells
# varies among 140 and 144
# key: nracks, value: ncells
cellconfigs = {2: 35, 3: 24, 4: 18, 5: 14, 6: 12, 7: 10, 8: 9,
               9: 8, 10: 7, 12: 6, 14: 5, 18: 4, 24: 3, 35: 2}

In [27]:
# number of points to sample
N = 200

df = pd.DataFrame(data={
    "t_cover": [random.uniform(0.5e-3, 2.0e-3) for _ in range(N)],
    "channel_width": [random.uniform(4e-3, 19e-3) for _ in range(N)], 
    "channel_height": [random.uniform(0.5e-3, 6e-3) for _ in range(N)],
    "nracks": random.choices(list(cellconfigs.keys()), k=N)
})

# compute ncells
df["ncells"] = df.nracks.apply(lambda nr: cellconfigs[nr])

# add constants
for k, v in constants.items():
    df[k] = v
    
# random m_flow (per rack)
df['m_flow'] = 120/3600/df['nracks'] * [random.uniform(0.1, 10) for _ in range(N)]
    
# drop rows with w < h
df.query("channel_width >= channel_height", inplace=True)
    
# compute modelica stuff
df["area_cover"] = df["channel_width"] * df["cell_width"]
df["t_plate"] = df["channel_dist"] - df["channel_width"]
df["area_plate"] = df["channel_height"] * df["cell_width"] #  + 2 * t_cover??
df["area_channel"] = df["channel_height"] * df["channel_width"]
df["perim_channel"] = 2 * (df["channel_height"] + df["channel_width"])

# create simulation id
df["simid"] = [str(uuid1()) for _ in range(len(df))]

In [28]:
df.describe().T

count         mean           std          min          25%  \
t_cover         196.0     0.001199  4.180864e-04     0.000508     0.000832   
channel_width   196.0     0.012184  4.309453e-03     0.004192     0.008513   
channel_height  196.0     0.003203  1.611862e-03     0.000542     0.001839   
nracks          196.0    10.816327  8.320931e+00     2.000000     5.000000   
ncells          196.0    11.193878  8.463688e+00     2.000000     5.000000   
C_quartercell   196.0    36.300000  1.424725e-14    36.300000    36.300000   
k_cell          196.0     3.311000  4.452264e-16     3.311000     3.311000   
c_cover         196.0   888.000000  0.000000e+00   888.000000   888.000000   
k_cover         196.0   220.000000  0.000000e+00   220.000000   220.000000   
rho_cover       196.0  2700.000000  0.000000e+00  2700.000000  2700.000000   
c_plate         196.0   888.000000  0.000000e+00   888.000000   888.000000   
k_plate         196.0   220.000000  0.000000e+00   220.000000   220.000000   
rho_plate       196.0  2700.000000  0.000000e+00  2700.000000  2700.000000   
channel_dist    196.0     0.040000  6.956663e-18     0.040000     0.040000   
cell_width      196.0     0.043000  0.000000e+00     0.043000     0.043000   
stoptime        196.0  2000.000000  0.000000e+00  2000.000000  2000.000000   
m_flow          196.0     0.026437  2.729330e-02     0.000324     0.006905   
area_cover      196.0     0.000524  1.853065e-04     0.000180     0.000366   
t_plate         196.0     0.027816  4.309453e-03     0.021003     0.024112   
area_plate      196.0     0.000138  6.931007e-05     0.000023     0.000079   
area_channel    196.0     0.000039  2.480986e-05     0.000004     0.000019   
perim_channel   196.0     0.030772  9.300659e-03     0.010752     0.023744   

                        50%          75%          max  
t_cover            0.001151     0.001557     0.001991  
channel_width      0.012807     0.015888     0.018997  
channel_height     0.003187     0.004529     0.005968  
nracks             8.000000    14.000000    35.000000  
ncells             9.000000    14.000000    35.000000  
C_quartercell     36.300000    36.300000    36.300000  
k_cell             3.311000     3.311000     3.311000  
c_cover          888.000000   888.000000   888.000000  
k_cover          220.000000   220.000000   220.000000  
rho_cover       2700.000000  2700.000000  2700.000000  
c_plate          888.000000   888.000000   888.000000  
k_plate          220.000000   220.000000   220.000000  
rho_plate       2700.000000  2700.000000  2700.000000  
channel_dist       0.040000     0.040000     0.040000  
cell_width         0.043000     0.043000     0.043000  
stoptime        2000.000000  2000.000000  2000.000000  
m_flow             0.016490     0.037741     0.166367  
area_cover         0.000551     0.000683     0.000817  
t_plate            0.027193     0.031487     0.035808  
area_plate         0.000137     0.000195     0.000257  
area_channel       0.000036     0.000055     0.000107  
perim_channel      0.031396     0.038467     0.048411

In [44]:
# save as csv
df.to_csv("../data/200_samples_Vdot_variable.csv", index=False)

In [95]:
# save as individual json files
for d in df.to_dict(orient='index').values():
    simid = d["simid"]
    with open("../data/modelica_input_%s.json"%simid, "w", encoding="utf-8") as fp:
        json.dump(d, fp, indent=2)

In [47]:
# save as individual datafiles for compiled openmodelica models
import os
datdir = "/home/oliver/projects/kiss_test"
bindir = "/home/oliver/projects/kiss_work"
for row in df.itertuples():
    s = ""
    for k in ['t_cover', 
               'C_quartercell', 'k_cell', 'c_cover', 'k_cover', 'rho_cover',
               'c_plate', 'k_plate', 'rho_plate',
               'm_flow', 'area_cover', 't_plate', 'area_plate',
               'area_channel', 'perim_channel']:
        s += f"{k}={getattr(row, k):.6g}\n"
    with open(os.path.join(datdir, f"{row.simid}.txt"), "w") as fp:
        fp.write(s)

In [48]:
# write a shell script for computations
n_jobs = 6
with open(os.path.join(bindir, "runall.sh"), "w") as fp:
    for i, row in enumerate(df.itertuples()):
        fpath = os.path.join(datdir, f"{row.simid}.txt")
        fp.write(f"./Kiss{row.nracks}x{row.ncells} -overrideFile {fpath} -r {row.simid}_res.mat -outputPath {datdir}")
        if (i+1)%n_jobs == 0:
            fp.write("\n")
        else:
            fp.write(" &\n")
        